In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split
import streamlit as st
import pickle
import os


In [2]:
from dotenv import load_dotenv
from alpha_vantage.timeseries import TimeSeries
from sklearn.preprocessing import StandardScaler


In [3]:
model=None
with open('model.pkl','rb') as f:
    model=pickle.load(f)


In [4]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100, 128)       │        69,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 363,461 (1.39 MB)

 Trainable params: 121,153 (473.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 242,308 (946.52 KB)

In [5]:
API_KEY=None
with open(".env.txt") as f:
    for line in f:
        if line.startswith("API_KEY"):
            API_KEY=line.strip().split("=")[1]
            break
print(API_KEY)

OGLHOIQJG5P5TBWV


In [6]:
def get_stock_data(symbol:str):
    ts=TimeSeries(key=API_KEY,output_format="pandas")
    data,_=ts.get_daily(symbol=symbol,outputsize="compact")
    data.reset_index(inplace=True)
    df=pd.DataFrame(data)
    return df
    

In [14]:
df=get_stock_data('RELIANCE.BSE')
print(df.head(5))
print(f'The shape is{df.shape}')
# print(f')
print(df.columns)
print(len(df.columns))

        date  1. open  2. high   3. low  4. close  5. volume
0 2025-11-12  1500.00  1524.00  1500.00   1511.75   854483.0
1 2025-11-11  1496.00  1498.70  1480.25   1493.90   214867.0
2 2025-11-10  1473.15  1500.00  1473.15   1489.25   408088.0
3 2025-11-07  1497.65  1498.50  1476.25   1478.25   378528.0
4 2025-11-06  1483.10  1501.95  1477.55   1495.75  1178418.0
The shape is(100, 6)
Index(['date', '1. open', '2. high', '3. low', '4. close', '5. volume'], dtype='object')
6


In [29]:
def preprocess(df):
    rename_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
    df.columns = rename_cols + list(df.columns[len(rename_cols):])  # preserve extras if present

    # add encoded symbol if missing
    if 'symbol_encoded' not in df.columns:
        df['symbol_encoded'] = 0
    # if 'target' not in df.columns:
    #     df['target'] = 0

    X_input = df[['open','high','low','close','volume','symbol_encoded']]

    scaler=None
    with open('Standard_scaler.pkl','rb') as f:
        scaler=pickle.load(f)
    
    X_scaled=scaler.transform(X_input.to_numpy())
        
    dummy_target = np.zeros((X_scaled.shape[0], 1))
    X_scaled = np.hstack([X_scaled, dummy_target])  # now 7 columns
    X_scaled = X_scaled.reshape(1, 100, 7)

    
    return X_scaled,scaler 

def modelPredict(model,X_scaled):
    
    pred_scaled = model.predict(X_scaled)
    predicted_close = float(pred_scaled[0][0])
    
    return predicted_close
       


In [30]:
X_scaled, scaler = preprocess(df)

# Predict
predicted_close = modelPredict(model, X_scaled)

# # Compare with last close
last_close = df['close'].iloc[-1]
pct_change = ((predicted_close - last_close) / last_close) * 100

print(f"Last close: {last_close:.2f}")
print(f"Predicted next close: {predicted_close:.2f}")
print(f"Change: {pct_change:+.2f}%")
# print(model.input_shape)
# print(X_scaled.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Last close: 1433.65
Predicted next close: 1439.07
Change: +0.38%


c:\Users\fortn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
